In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pylab
import glob
from pyds9 import *

# Astropy Packages
import astropy.io.fits as pf
from astropy.convolution import Gaussian2DKernel
from astropy.stats import gaussian_fwhm_to_sigma
from astropy.stats import sigma_clipped_stats
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize

from photutils import detect_sources
from photutils import detect_threshold
from photutils import find_peaks
from photutils import detection



#-----------------------------------------------------------------------------------------
'''

pyCandClick: a Python implementation of Rob De Rosa's IDL cand_click.pro routine to measure fluxes for multiple stars in a FITS image and estimate their magnitude differences. Also includes automated source detection and centroiding from the photutils package.

Dependencies include the following (some can be installed via pip):
- AstroPy library, including photutils: http://www.astropy.org/
- SAO ds9: http://ds9.si.edu/site/Home.html
- pyds9: https://github.com/ericmandel/pyds9

To run, use the following command: 
python pyCandClick.py

(v1) 12/17/2015 - K. Ward-Duong: Created script.

'''
#-----------------------------------------------------------------------------------------

In [ ]:
#-----------------------------------------------------------------------------------------
# Define variables here:
platescale = 0.02 # arcsec/px
wavelength = 2.23E-6 # meters; make this more intelligent to grab filter from FITS header
diameter = 6.5 # meters
fwhm = (1.22*wavelength/diameter*206265)/platescale # arcsec converted into pixels

In [2]:
#-----------------------------------------------------------------------------------------
# Define functions here:

# Automatically identify stars
def starfinder(data):
    # Detect the source peaks at a 3-sigma level and put into table (tbl):
    mean, median, std = sigma_clipped_stats(data, sigma=3.0)
    threshold = median + (3.0 * std)
    tbl = find_peaks(data, threshold)
    return tbl

# Get user input to select the brightest (primary) star:
def onclick(event):
    if event.xdata != None and event.ydata != None:
        print(event.xdata, event.ydata)
        return event.xdata, event.ydata
        
        
def on_key(event, arg1, arg2, arg3):
    if event.xdata != None and event.ydata != None:
        print('you pressed', event.key, event.xdata, event.ydata)
        
def getphot(positions, radius, data):
    apertures = CircularAperture(positions, r = radius)
    phot_table = aperture_photometry(data, apertures)
    
def starfinder_iraf():
    detection.irafstarfind(data, threshold, fwhm, minsep_fwhm=0.5, sky=None, exclude_border=False)


#-----------------------------------------------------------------------------------------

In [3]:
# Get all of the FITS files in the directory of your choice:
files = glob.glob('/Users/Kim/Research/ExoplanetHostProject/mm_final_reduction/HAT-P-39/*.fits')

# Open a ds9 window and set the frame
#d = DS9()
#d.set('frame 1')
    
# Loop over each of the FITS files:
for ii in xrange(0, len(files)): 
    name = files[ii]
    
    # Get the data and header from the ii'th FITS file
    data, header = pf.getdata(name, header = True)
    star_positions = starfinder(data)
    
    # Display to ensure appropriate stars are found
    plt.clf(), plt.cla() # clear any previous figures and axes
    ax, fig = plt.gca(), plt.gcf()
    norm = ImageNormalize(stretch=SqrtStretch()) # normalize image stretch
    
    print "Click to select the primary star: "
    #implot = ax.imshow(data, cmap='Greys_r', origin='lower', norm=ImageNormalize(stretch=SqrtStretch())) # normalize image stretch when loading data
    plt.imshow(data, cmap='Greys_r', origin='lower', norm=ImageNormalize(stretch=SqrtStretch())) # normalize image stretch when loading data
    plt.plot(star_positions[:,1], star_positions[:,0], ls='none', color='cyan', marker='+', ms=10, lw=1.5) # plot peaks, note swapped x/y!
    
    x = pylab.ginput(2)
    pylab.imshow(x)
    
    # Get user input to select the brightest (primary) star:
    #cid = fig.canvas.mpl_connect('button_press_event', onclick)
    #cid = fig.canvas.mpl_connect('key_press_event', lambda event: on_key(event, plt1, plt2, plt3))
    plt.show()
    

Click to select the primary star: 


NameError: name 'cid' is not defined